# Evalution of Replicate Agreement
In this notebook, I want to explore how well the replicates of the dual RNA-seq experiements agree with one another.
For that purpose, I'll load three different runs of the GCN for the three different replicates.
Then, I'll see how much they have in common.

In [1]:
import numpy as np
import pandas as pd
import h5py, os

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn

from statsmodels.stats.inter_rater import fleiss_kappa, cohens_kappa
%matplotlib inline

/home/sasse/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Loading GCN Predictions

In [3]:
model_dir_rep1 = '../data/GCN/training/legionella_rep1_cv5_good/'
model_dir_rep2 = '../data/GCN/training/legionella_rep2_cv5_good/'
model_dir_rep3 = '../data/GCN/training/legionella_rep3_cv5_good/'
data_file = '../data/preprocessing/legionella_gcn_input_unbalanced.h5'
CLASSIFICATION_THRESHOLD = 0.5

with h5py.File(data_file, 'r') as f:
    network = f['network'][:]
    features = f['features'][:]
    node_names = f['gene_names'][:]
    y_train = f['y_train'][:]
    y_test = f['y_test'][:]
    if 'y_val' in f:
        y_val = f['y_val'][:]
    else:
        y_val = None
    train_mask = f['mask_train'][:]
    test_mask = f['mask_test'][:]
    if 'mask_val' in f:
        val_mask = f['mask_val'][:]
    else:
        val_mask = None

# get a label dataframe
labels_df = pd.DataFrame(node_names, index=node_names[:, 0], columns=['ID', 'name']).drop('ID', axis=1)
labels_df['label'] = (y_train[:,0] | y_test[:,0])
labels_df['train_label'] = y_train[:, 0]
labels_df['test_label'] = y_test[:, 0]


def prediction_stats(predictions, labels_df):
    labels_with_pred = labels_df.join(predictions)
    predictions_for_knowns = labels_with_pred[labels_with_pred.label == 1]
    pos_pred = predictions_for_knowns[predictions_for_knowns.Prob_pos > CLASSIFICATION_THRESHOLD]
    known_pos_predictions = pos_pred.shape[0]
    number_of_knowns = predictions_for_knowns.shape[0]
    pred_test = pos_pred[pos_pred.test_label == 1]
    pred_train = pos_pred[pos_pred.train_label == 1]

    print ("Predicted {} out of {} known infection genes".format(known_pos_predictions,
                                                                 number_of_knowns)
          )
    print ("Out of these {}, {} were not shown during training.".format(number_of_knowns,
                                                                        y_test[:, 0].sum())
          )
    print ("Predicted {} out of {} test genes ({}%)".format(pred_test.shape[0],
                                                            y_test[:,0].sum(),
                                                            pred_test.shape[0]/y_test[:,0].sum()*100.)
          )
    print ("Predicted {} out of {} train genes ({}%)".format(pred_train.shape[0],
                                                             y_train[:,0].sum(),
                                                             pred_train.shape[0]/y_train[:,0].sum()*100.)
          )
    predictions_for_knowns.sort_values(by='Prob_pos', ascending=False)
    return predictions_for_knowns

def load_predictions(model_dir):
    predictions = []
    for file in os.listdir(model_dir):
        if file.startswith('predictions_'):
            df = pd.read_csv(os.path.join(model_dir, file), sep='\t')
            predictions.append(df)
    return predictions

def load_hdf_data(path, feature_name='features'):
    with h5py.File(path, 'r') as f:
        network = f['network'][:]
        features = f[feature_name][:]
        node_names = f['gene_names'][:]
        y_train = f['y_train'][:]
        y_test = f['y_test'][:]
        if 'y_val' in f:
            y_val = f['y_val'][:]
        else:
            y_val = None
        train_mask = f['mask_train'][:]
        test_mask = f['mask_test'][:]
        if 'mask_val' in f:
            val_mask = f['mask_val'][:]
        else:
            val_mask = None
    return network, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, node_names

def get_consensus(predictions):
    all_nodes = list(predictions[0].ID)
    pos_preds = [list(p[p.Prob_pos >= CLASSIFICATION_THRESHOLD].ID) for p in predictions]
    return set(all_nodes).intersection(*pos_preds)

def avg_rep_score(predictions):
    avg_df = predictions[0]
    avg_df.columns = ['ID', 'Name', 'Prob_pos_cv_0']
    cv_run = 1
    means = avg_df.Prob_pos_cv_0
    for p in predictions[1:]:
        avg_df['Prob_pos_cv_{}'.format(cv_run)] = p.Prob_pos
        means = means + p.Prob_pos
        cv_run += 1
    means /= len(predictions)
    avg_df['mean_score'] = means
    return avg_df

def replicate_agreement(df1, df2, df3, n):
    pass

## Basic Statistics over Replicates

In [4]:
model_dir_rep1 = '../data/GCN/training/legionella_rep1_cv5_good/'
model_dir_rep2 = '../data/GCN/training/legionella_rep2_cv5_good/'
model_dir_rep3 = '../data/GCN/training/legionella_rep3_cv5_good/'

all_cons_preds = []
rep_no = 1
for model_dir in [model_dir_rep1, model_dir_rep2, model_dir_rep3]:
    preds = load_predictions(model_dir)
    no_predicted = [df[df.Prob_pos >= .5].shape[0] for df in preds]
    consensus = get_consensus(preds)
    print ("Replicate {}: {} CV runs agree on {} genes".format(rep_no, len(preds), len(consensus)))
    print ("Predicted # of genes: {}".format(no_predicted))
    all_cons_preds.append(consensus)
    rep_no += 1

rep_agreement_all = set(all_cons_preds[0].intersection(*all_cons_preds))
print ("All replicates agreeing on {} genes.".format(len(rep_agreement_all)))

Replicate 1: 5 CV runs agree on 687 genes
Predicted # of genes: [983, 817, 925, 847, 905]
Replicate 2: 5 CV runs agree on 573 genes
Predicted # of genes: [776, 695, 748, 720, 722]
Replicate 3: 5 CV runs agree on 789 genes
Predicted # of genes: [934, 1011, 982, 1023, 1001]
All replicates agreeing on 341 genes.


In [5]:
# get DF with positives for model 1
preds = load_predictions(model_dir_rep1)
df = avg_rep_score(preds)
rep_1 = df[df > .5].dropna().sort_values(by='mean_score', ascending=False)

In [6]:
# get DF with positives for model 2
preds = load_predictions(model_dir_rep2)
df = avg_rep_score(preds)
rep_2 = df[df > .5].dropna().sort_values(by='mean_score', ascending=False)

In [7]:
# get DF with positives for model 3
preds = load_predictions(model_dir_rep3)
df = avg_rep_score(preds)
rep_3 = df[df > .5].dropna().sort_values(by='mean_score', ascending=False)

In [31]:
all_reps_joined = rep_1.merge(rep_2, on='ID', suffixes=('_rep1', '_rep2')).merge(rep_3, on='ID', suffixes=('', '_rep3'))
all_reps_joined.drop(['Name_rep1', 'Name_rep2'], axis=1, inplace=True)
gene_names = all_reps_joined['Name']
all_reps_joined.drop('Name', axis=1, inplace=True)
all_reps_joined.insert(1, 'Name', gene_names)
all_reps_joined.to_csv('../data/replicate_agreement/replicate_agreement.tsv', sep='\t')
all_reps_joined

ID      Name  Prob_pos_cv_0_rep1  Prob_pos_cv_1_rep1  \
0    ENSG00000132002    DNAJB1               0.984               0.969   
1    ENSG00000146233   CYP39A1               0.983               0.971   
2    ENSG00000198576       ARC               0.995               0.955   
3    ENSG00000013588    GPRC5A               0.988               0.962   
4    ENSG00000184545     DUSP8               0.988               0.948   
5    ENSG00000119630       PGF               0.978               0.949   
6    ENSG00000123358     NR4A1               0.982               0.953   
7    ENSG00000141448     GATA6               0.982               0.950   
8    ENSG00000187140     FOXD3               0.982               0.954   
9    ENSG00000100852   ARHGAP5               0.981               0.946   
10   ENSG00000135111      TBX3               0.984               0.941   
11   ENSG00000142871     CYR61               0.988               0.935   
12   ENSG00000162616    DNAJB4               0.983               0.944   
13   ENSG00000172602      RND1               0.987               0.943   
14   ENSG00000134363       FST               0.981               0.944   
15   ENSG00000104369      JPH1               0.976               0.928   
16   ENSG00000087494     PTHLH               0.989               0.932   
17   ENSG00000198765     SYCP1               0.978               0.955   
18   ENSG00000139926     FRMD6               0.987               0.923   
19   ENSG00000119508     NR4A3               0.946               0.960   
20   ENSG00000148677    ANKRD1               0.975               0.951   
21   ENSG00000142178      SIK1               0.980               0.941   
22   ENSG00000115380    EFEMP1               0.973               0.921   
23   ENSG00000164379     FOXQ1               0.979               0.951   
24   ENSG00000120885       CLU               0.987               0.932   
25   ENSG00000127152    BCL11B               0.975               0.938   
26   ENSG00000183655    KLHL25               0.970               0.949   
27   ENSG00000127528      KLF2               0.973               0.953   
28   ENSG00000204099      NEU4               0.967               0.958   
29   ENSG00000125735   TNFSF14               0.966               0.946   
..               ...       ...                 ...                 ...   
311  ENSG00000163235      TGFA               0.763               0.597   
312  ENSG00000170989     S1PR1               0.672               0.754   
313  ENSG00000130522      JUND               0.657               0.671   
314  ENSG00000163661      PTX3               0.589               0.735   
315  ENSG00000165006     UBAP1               0.715               0.630   
316  ENSG00000175745     NR2F1               0.686               0.673   
317  ENSG00000178913      TAF7               0.739               0.637   
318  ENSG00000122644     ARL4A               0.760               0.589   
319  ENSG00000146457      WTAP               0.543               0.732   
320  ENSG00000188295    ZNF669               0.720               0.633   
321  ENSG00000075461    CACNG4               0.816               0.609   
322  ENSG00000124225    PMEPA1               0.652               0.525   
323  ENSG00000186416      NKRF               0.609               0.675   
324  ENSG00000140395     WDR61               0.511               0.672   
325  ENSG00000008405      CRY1               0.789               0.551   
326  ENSG00000176170     SPHK1               0.730               0.595   
327  ENSG00000172000    ZNF556               0.753               0.658   
328  ENSG00000152527   PLEKHH2               0.745               0.535   
329  ENSG00000087074  PPP1R15A               0.687               0.591   
330  ENSG00000125945    ZNF436               0.664               0.613   
331  ENSG00000091831      ESR1               0.783               0.532   
332  ENSG00000112186      CAP2               0.666               0.586   
333  ENSG00000177707  

## Similarity of Predictions
A lot of research has been done already to measure the extend to which different raters agree on an observation.
This is called **inter-rater reliability** and can be used here to determine how well the predictions for the three replicates differ.

### Fleiss' Kappa Statistics
That  is a measure usable for categorical predictions and any number of raters. It's thus perfect for me.

In [5]:
all_pos = pd.concat([pred_rep1.Prob_pos > CLASSIFICATION_THRESHOLD,
                     pred_rep2.Prob_pos > CLASSIFICATION_THRESHOLD,
                     pred_rep3.Prob_pos > CLASSIFICATION_THRESHOLD], axis=1)
all_pos.columns = ['predicted_rep1', 'predicted_rep2', 'predicted_rep3']
all_neg = pd.concat([pred_rep1.Prob_pos <= CLASSIFICATION_THRESHOLD,
                     pred_rep2.Prob_pos <= CLASSIFICATION_THRESHOLD,
                     pred_rep3.Prob_pos <= CLASSIFICATION_THRESHOLD], axis=1)
all_neg.columns = ['not_predicted_rep1', 'not_predicted_rep2', 'not_predicted_rep3']
count_table = pd.concat([all_pos.sum(axis=1), all_neg.sum(axis=1)], axis=1)
count_table.columns = ['Num_Pos', 'Num_Neg']
print ("Fleiss' Kappa Value: {}".format(fleiss_kappa(count_table)))

Fleiss' Kappa Value: 0.43914760925238777


### Agreement Statistics
Just print some basic statistics as to how many genes are predicted by all replicates and how many replicates make the same prediction. Also state the number of deviating genes and which of the replicates is most responsible for that.

In [30]:
all_reps_joined.corr()

Prob_pos_cv_0_rep1  Prob_pos_cv_1_rep1  \
Prob_pos_cv_0_rep1               1.000               0.766   
Prob_pos_cv_1_rep1               0.766               1.000   
Prob_pos_cv_2_rep1               0.907               0.872   
Prob_pos_cv_3_rep1               0.844               0.821   
Prob_pos_cv_4_rep1               0.839               0.881   
mean_score_rep1                  0.916               0.921   
Prob_pos_cv_0_rep2               0.355               0.426   
Prob_pos_cv_1_rep2               0.447               0.469   
Prob_pos_cv_2_rep2               0.404               0.462   
Prob_pos_cv_3_rep2               0.352               0.444   
Prob_pos_cv_4_rep2               0.462               0.468   
mean_score_rep2                  0.423               0.476   
Prob_pos_cv_0                    0.697               0.610   
Prob_pos_cv_1                    0.680               0.658   
Prob_pos_cv_2                    0.698               0.668   
Prob_pos_cv_3                    0.675               0.683   
Prob_pos_cv_4                    0.711               0.637   
mean_score                       0.726               0.682   

                    Prob_pos_cv_2_rep1  Prob_pos_cv_3_rep1  \
Prob_pos_cv_0_rep1               0.907               0.844   
Prob_pos_cv_1_rep1               0.872               0.821   
Prob_pos_cv_2_rep1               1.000               0.928   
Prob_pos_cv_3_rep1               0.928               1.000   
Prob_pos_cv_4_rep1               0.921               0.940   
mean_score_rep1                  0.975               0.957   
Prob_pos_cv_0_rep2               0.394               0.400   
Prob_pos_cv_1_rep2               0.451               0.469   
Prob_pos_cv_2_rep2               0.427               0.430   
Prob_pos_cv_3_rep2               0.339               0.342   
Prob_pos_cv_4_rep2               0.500               0.536   
mean_score_rep2                  0.443               0.456   
Prob_pos_cv_0                    0.712               0.750   
Prob_pos_cv_1                    0.703               0.674   
Prob_pos_cv_2                    0.706               0.669   
Prob_pos_cv_3                    0.676               0.618   
Prob_pos_cv_4                    0.723               0.721   
mean_score                       0.738               0.721   

                    Prob_pos_cv_4_rep1  mean_score_rep1  Prob_pos_cv_0_rep2  \
Prob_pos_cv_0_rep1               0.839            0.916               0.355   
Prob_pos_cv_1_rep1               0.881            0.921               0.426   
Prob_pos_cv_2_rep1               0.921            0.975               0.394   
Prob_pos_cv_3_rep1               0.940            0.957               0.400   
Prob_pos_cv_4_rep1               1.000            0.968               0.399   
mean_score_rep1                  0.968            1.000               0.418   
Prob_pos_cv_0_rep2               0.399            0.418               1.000   
Prob_pos_cv_1_rep2               0.459            0.486               0.866   
Prob_pos_cv_2_rep2               0.439            0.458               0.942   
Prob_pos_cv_3_rep2               0.358            0.390               0.818   
Prob_pos_cv_4_rep2               0.487            0.518               0.881   
mean_score_rep2                  0.449            0.476               0.945   
Prob_pos_cv_0                    0.711            0.733               0.426   
Prob_pos_cv_1                    0.640            0.708               0.430   
Prob_pos_cv_2                    0.652            0.716               0.388   
Prob_pos_cv_3                    0.645            0.697               0.456   
Prob_pos_cv_4                    0.691            0.734               0.429   
mean_score                       0.701            0.753               0.446   

                    Prob_pos_cv_1_rep2  Prob_pos_cv_2_rep2  \
Prob_pos_cv_0_rep1               0.447               0.404   
Prob_pos_cv_1_rep1               0.4

In [10]:
fig = plt.figure(figsize=(14, 8))
#plt.hist(count_table.Num_Pos, bins=np.arange(0, 4, 1))
count_table.Num_Pos.plot(kind='hist')